# Teams and match classes

In [2]:
import numpy as np
import nbimporter
from players import PlayerCreator, Player
from collections import defaultdict
import time
import matplotlib.pyplot as plt
import imageio
from IPython.display import clear_output

In [3]:
class Team(object):
    
    def __init__(self, name, data):
        self.name = name
        self.roster = [
            PlayerCreator.create(row) for i, row in 
            data[data['club'] == self.name].iterrows()
        ]
        self.team = defaultdict(lambda: [])
        self.players = []
        self.active_player = None
        
    def selection(self, module=None):
        self.team = defaultdict(lambda: [])
        if module is None:
            module = {Player.DEF: 4, Player.MID: 4, Player.ATT: 2}
        module[Player.GK] = 1
        options = sorted(self.roster, key=lambda x: -x.overall)
        for player in options:
            if module[player.role] > 0:
                self.team[player.role].append(player)
                self.players.append(player)
                module[player.role] -= 1
    
    @property
    def gk(self):
        try:
            return self.team[Player.GK][0]
        except IndexError:
            return None
        
    def show(self, ax, pos, c='#ffffff'):
        player_pos = {}
        p_map = {Player.GK: pos[0], Player.DEF: pos[1], Player.MID: pos[2], Player.ATT: pos[3]}
        x = []
        y = []
        for role, players in self.team.items():
            for i, p in enumerate(players):
                x.append(p_map[p.role])
                if len(players) > 1:
                    y_l = i + 5.5 / len(players)
                else:
                    y_l = 3
                y.append(y_l)
                player_pos[p.id] = (p_map[p.role], y_l)
        ax.scatter(x, y, c=c, s=300)
        return player_pos
        
    
    def print_selection(self):
        print (self.name)
        print (list(x.name for x in self.team[Player.GK]))
        print (list(x.name for x in self.team[Player.DEF]))
        print (list(x.name for x in self.team[Player.MID]))
        print (list(x.name for x in self.team[Player.ATT]))
        

class Match(object):
    
    opponents = {Player.GK: [Player.ATT], Player.DEF: [Player.ATT],
                Player.MID: [Player.MID], Player.ATT: [Player.DEF]}
    
    def __init__(self, home, away, duration=90):
        self.h = home
        self.a = away
        self.d = duration
        self.possession, self.defender = self.h, self.a
        self.possession.active_player = np.random.choice(
            self.possession.team[Player.MID])
        self.score = {self.h.name: 0, self.a.name: 0}
        self.scorers = []
    
    def kickoff(self, verbose=False, sleep=0, show=False):
        log = []
        for i in range(1, self.d + 1):
            if show:
                clear_output(wait=True)
                self.show()
            l = self.turn(i)
            log += l
            if verbose:
                print ("\n".join(l))
                time.sleep(sleep)
    
    @staticmethod
    def _select_opponent(active_player, passive_team):
        candidates = []
        for role in Match.opponents[active_player.role]:
            candidates += passive_team.team[role]
        return np.random.choice(candidates)
    
    def turn(self, i):
        log = []
        p = self.possession
        action, outcome, receiver = p.active_player.play(p.players)
        log.append("min {}: {} is in possession".format(i, p.active_player.name))
        if action == Player.SHOOT:
            log.append("min {}: SHOOT of {}".format(i, p.active_player.name))
            if outcome:
                if self.defender.gk.saves():
                    l = "min {}: {} saves on {}".format(i, self.defender.gk.name,
                                                       p.active_player.name)
                    self.possession = self.defender
                    self.defender = p
                    self.possession.active_player = self.possession.gk
                else:
                    l = "min {}: GOAL! {} scores for {}".format(i, p.active_player.name,
                                                               p.name)
                    self.scorers.append((i, p.active_player))
                    self.score[p.name] += 1
                    self.possession = self.defender
                    self.defender = p
                    self.possession.active_player = np.random.choice(
                        self.possession.team[Player.MID])
            else:
                l = "min {}: {} shoots out".format(i, p.active_player.name)
                self.possession = self.defender
                self.defender = p
                self.possession.active_player = self.possession.gk
        elif action == Player.PASS:
            opponent = Match._select_opponent(p.active_player, self.defender)
            if outcome:
                if opponent.defend():
                    l = "min {}: {} intercept passage of {}".format(i, 
                                                                    opponent.name,
                                                                    p.active_player.name)
                    self.possession = self.defender
                    self.defender = p
                    self.possession.active_player = opponent
                else:
                    l = "min {}: {} passes to {}".format(i, p.active_player.name,
                                                        receiver.name)
                    self.possession.active_player = receiver
            else:
                l = "min {}: {} fails passing to {}".format(i, p.active_player.name,
                                                        receiver.name)
                self.possession = self.defender
                self.defender = p
                self.possession.active_player = opponent
        else:
            pass
        log.append(l)
        return log
    
    def show(self):
        pitch = imageio.imread('../data/pitch.jpg')
        fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16, 8))
        axes.imshow(pitch, zorder=0, extent=[0, 9, 0, 6]) # left, right, bottom, top
        hP = self.h.show(axes, [1, 2, 4, 6], c='#000000')
        aP = self.a.show(axes, [8, 7, 5, 3], c='#660000')
        try:
            x, y = hP[self.possession.active_player.id]
        except KeyError:
            x, y = aP[self.possession.active_player.id]
        axes.scatter(x, y, c='#ffffff', s=60)
        plt.show()